# Resources
* https://www.kaggle.com/code/shrutimechlearn/step-by-step-diabetes-classification-knn-detailed
* https://www.kaggle.com/code/ash316/ml-from-scratch-part-2
* https://www.kaggle.com/code/pouryaayria/a-complete-ml-pipeline-tutorial-acu-86 (Promising to implement)
* https://www.kaggle.com/code/vincentlugat/pima-indians-diabetes-eda-prediction-0-906
* https://www.kaggle.com/code/faysalmiah1721758/pima-indians-diabetes
* https://www.kaggle.com/code/walidkw/pima-indians-diabetes-ml-model-selection-83 (Promising to implement)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

# Introduction and data analysis 
Describe the problem being addressed. Provide a detailed characterization of the task dataset in terms of format, volume, quality and bias.

* Problem Description: 
> Start by introducing the problem or research question that your analysis aims to address. Clearly define the scope and objectives of your analysis. Explain why this problem is important or relevant.

* Dataset Description:

> Format: 
>> Describe the format of the dataset. Is it structured (e.g., CSV, Excel) or unstructured (e.g., text, images)? Mention the data types present (e.g., numerical, categorical). 

> Volume: 
>> Specify the size of the dataset in terms of the number of records and features (columns). Mention if it's a small, medium, or large dataset.

> Quality: 
>> Discuss the quality of the dataset. Are there missing values, outliers, or data errors? How were these issues handled (e.g., data imputation, outlier removal)?

> Bias: 
>> Address any potential biases in the dataset. Bias can arise from data collection methods, sampling, or other factors. Describe how bias was considered and handled, if applicable.

## Format:
The Pima Indian Diabetics Dataset is in a structured tabular format. The dataset is available on Kaggle in the CSV (Comma-Seperated-Values) file format. The dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. Each data entries or rows in the dataset represents an individual, whereas each columns represents attributes related to the subject health like glucose level, blood pressure, etc. For the features, most of them are integer values with exception to "BMI" and "DiabetesPedigreeFunction" that are floating-point value.

TALK ABOUT THE OBJECTIVE OF THE DATASET SOMEWHERE?

## Volume:
The dataset consisted of 768 instances, which is considerd to be moderate number of subjects. Each instance comes with a set of medical measurements (like including glucose level, blood pressure, and BMI) and a target varialbe called Outcome that indicate whether the subject has diabetic or not. If the value in the Outcome column reads 0, then the person doesn't have diabetics; otherwise, the person with diabetic will have Outcome column reads 1. 

## Quality:
Given the statistical analysis, a very intriguing question arises. Is it possible for the columns including "Glucose", "BloodPressure", "SkinThickness", "Insulin", and "BMI" to have minimum value of zero? In this setting, the value of zero doesn't make sense. For example, a person with BMI means a person weight zero kilogram, which is impossible. We speculate that these instances were treated as missing value (replace with zero) or recording error from human mannual data entry. In order to address this problem, we will considered replacing the zero values with either mean or median of the associated column. 

We used boxplot to investigate the outlier in the dataset. Using the IQR (Inter-qaurtile Range) outlier detection approach, the data that shown as dot on the boxplot is considered as outlier. We also put multiple text annotations to indicate the number of outlier for each feature. 

HOW TO ADDRESS THE OUTLIER BEFORE TRAINING THE MODEL?

## Bias:
The bias may arise from the sampling process, data collection methods, and population representation. For instance, the dataset includes specific group of people, the Pima Indians; therefore, the analysis and model's applicability may not be suitable for other population. Henceforth, the prediction model is generalized enough to apply with other diverse setting. 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# To make sure that there are only two quanitiy in the Outcome (y)
df['Outcome'].unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

outcome_counts = df['Outcome'].value_counts()

total_count = outcome_counts.sum()

plt.figure(figsize=(6, 5))
bars = plt.bar(x=[0, 1], height=outcome_counts, color=['blue', 'red'], alpha=0.7)
plt.title('Frequency of Test Outcome (Positive vs. Negative)')
plt.xlabel('Outcome')
plt.ylabel('Frequency')
plt.xticks([0, 1], ['Negative Tested (0)', 'Positive Tested (1)'], rotation=0)

# Customize the labels inside the bars
for i, (count, percentage) in enumerate(zip(outcome_counts, outcome_counts / total_count * 100)):
    plt.text(i, count / 2, f'{count} subjects\n({percentage:.2f}%)', ha='center', va='center', 
             fontsize=12, fontweight='bold', color='white')

plt.show()

In [ ]:
num_columns = len(df.drop(['Outcome'],axis=1).columns)
num_rows = (num_columns // 3) + (num_columns % 3)

fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(10, 6 * num_rows // 2))
axes = axes.flatten()

for i, col in enumerate(df.drop(['Outcome'],axis=1).columns):
    ax = axes[i]
    boxplot = df[col].plot(kind='box', ax=ax, sharex=False, sharey=False, 
                           meanline=True, showmeans =True, meanprops={'color': 'red', 'linestyle': '--'})
    ax.set_title(f'{col}')
    ax.set_xticks([])

    # Calculate the median and mean
    median, mean = df[col].median(), df[col].mean() 
    
    # Mean and Median Annotation
    ax.text(0.62, 0.9, f'Median: {median:.2f}', transform=ax.transAxes, 
            fontsize=9, verticalalignment='top',color='green')
    ax.text(0.62, 0.8, f'Mean: {mean:.2f}', transform=ax.transAxes, 
            fontsize=9, verticalalignment='top',color='red')

    # Outlier calculation and Annotation
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    ax.text(0.62, 0.7, f'Outliers: {len(outliers)}', transform=ax.transAxes, fontsize=9, verticalalignment='top')

for i in range(num_columns, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()


In [ ]:
num_features = len(df.columns) - 1  
num_rows,num_cols = 3, 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8 * num_rows // 2))
axes = axes.flatten()

for i, column in enumerate(df.columns[:-1]):
    ax = axes[i]
    # Plot All the data points
    sns.histplot(data=df, x=column, bins=20, common_norm=False, ax=ax, 
                 legend=False, color='orange', alpha=0.2, edgecolor='none')
    
    # Plot the data points seperated by hue (Outcome)
    sns.histplot(data=df, x=column, hue="Outcome", bins=20, common_norm=False, kde=True,
                 ax=ax, legend=False, palette={0: 'blue', 1: 'red'}, alpha=0.4, edgecolor='none')
    
    mean = df[column].mean()
    var = df[column].var()
    skew = df[column].skew()
    
    ax.set_title(f'{column}\n(Mean={mean:.2f}, Var={var:.2f}, Skew={skew:.2f})')
    ax.set_xlabel(column)
    ax.set_ylabel('Frequency')
    
    # Calculate seperate mean and variance 
    mean_0 = df[df['Outcome'] == 0][column].mean()
    mean_1 = df[df['Outcome'] == 1][column].mean()
    var_0 = df[df['Outcome'] == 0][column].std()
    var_1 = df[df['Outcome'] == 1][column].std()

    ax.annotate(f'Mean(0): {mean_0:.2f}', xy=(0.7125, 0.85), xycoords='axes fraction', fontsize=10, color='blue')
    ax.annotate(f'Mean(1): {mean_1:.2f}', xy=(0.7125, 0.75), xycoords='axes fraction', fontsize=10, color='red')
    ax.annotate(f'Std(0): {var_0:.2f}', xy=(0.7125, 0.65), xycoords='axes fraction', fontsize=10, color='blue')
    ax.annotate(f'Std(1): {var_1:.2f}', xy=(0.7125, 0.55), xycoords='axes fraction', fontsize=10, color='red')
    
    ax.axvline(mean, color='black', linestyle='-.', linewidth=1.5)
    ax.axvline(df[df['Outcome'] == 0][column].mean(), color='blue', linestyle='--', linewidth=1.5)
    ax.axvline(df[df['Outcome'] == 1][column].mean(), color='red', linestyle='--', linewidth=1.5)

for i in range(num_features, num_rows * num_cols):
    fig.delaxes(axes[i])

custom_legend = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10)
]
legend_labels = ['All the Data', 'Negative Tested', 'Positive Tested']
legend = fig.legend(custom_legend, legend_labels, loc='lower right', bbox_to_anchor=(1.0, 0.0))
legend.set_bbox_to_anchor((0.90, 0.15))

plt.tight_layout()
plt.show()

In [ ]:
num_rows, num_cols = 3, 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8 * num_rows // 2))
axes = axes.flatten()

for i, column in enumerate(df.columns[:-1]):
    ax = axes[i]

    # Plot the KDE (Kernel Density Estimation) plot for the entire dataset
    sns.kdeplot(data=df, x=column, ax=ax, color='orange', legend=False, linewidth=2, alpha=0.6)

    # Plot the KDE plot for data points separated by hue (Outcome)
    sns.kdeplot(data=df[df['Outcome'] == 0], x=column, ax=ax, color='blue', label='Negative Tested', linewidth=2, alpha=0.6)
    sns.kdeplot(data=df[df['Outcome'] == 1], x=column, ax=ax, color='red', label='Positive Tested', linewidth=2, alpha=0.6)
    
    # From the previous graph
    mean = df[column].mean()
    var = df[column].var()
    skew = df[column].skew()
    
    ax.set_title(f'{column}\n(Mean={mean:.2f}, Var={var:.2f}, Skew={skew:.2f})')
    ax.set_xlabel(column)
    ax.set_ylabel('Density')
    
    # Calculate seperate mean and variance 
    mean_0 = df[df['Outcome'] == 0][column].mean()
    mean_1 = df[df['Outcome'] == 1][column].mean()
    std_0 = df[df['Outcome'] == 0][column].std()
    std_1 = df[df['Outcome'] == 1][column].std()

    ax.annotate(f'Mean(0): {mean_0:.2f}', xy=(0.7125, 0.85), xycoords='axes fraction', fontsize=10, color='blue')
    ax.annotate(f'Mean(1): {mean_1:.2f}', xy=(0.7125, 0.75), xycoords='axes fraction', fontsize=10, color='red')
    ax.annotate(f'Std(0): {std_0:.2f}', xy=(0.7125, 0.65), xycoords='axes fraction', fontsize=10, color='blue')
    ax.annotate(f'Std(1): {std_1:.2f}', xy=(0.7125, 0.55), xycoords='axes fraction', fontsize=10, color='red')

    

# Remove remaining empty subplots
for i in range(num_features, num_rows * num_cols):
    fig.delaxes(axes[i])

handles, labels = ax.get_legend_handles_labels()
custom_legend = [
    plt.Line2D([0], [0], color='orange', lw=2, label='All the Data'),
    *handles 
]
legend_labels = ['All the Data', *labels]
legend = fig.legend(custom_legend, legend_labels, loc='lower right', bbox_to_anchor=(1.0, 0.0))
legend.set_bbox_to_anchor((0.90, 0.15))

plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df, hue='Outcome', diag_kind='kde')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
hmap = sns.heatmap(df.corr(), annot=True, cmap='RdBu', linewidths=3.5, linecolor='white')

# Add a title
plt.title("Pearson Correlation between Features")

hmap.set_xticklabels(hmap.get_xticklabels(), rotation=90, fontsize=9)
hmap.set_yticklabels(hmap.get_yticklabels(), fontsize=9)

# Customize the legend
cbar = hmap.collections[0].colorbar
cbar.set_label('Correlation', fontsize=10)
cbar.ax.tick_params(labelsize=10)

# Add additional artistic elements
plt.tight_layout()
plt.show()

# Data Manipulation
* Remove outliers from the dataset 
* Replace zero values in columns that doesn't make sense using mean or median
* Upsampling to handle data imbalance between Positive and Negative Dataset (ex. SMOT?)

### Remove Outlier from the Dataset

# Training a Machine Learning and Deep Learning Model
* Implement fives Algorithm to predict 
* Search model from the resources provided above (not limited to Deep Learning Model)

# Machine Learning Model

### Others

In [ ]:
# Machine learning and statistical
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, KFold

scaler = StandardScaler()
X =  pd.DataFrame(scaler.fit_transform(df.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

classifiers = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('XGBoost', XGBClassifier(random_state=42)),
    ('SVM', SVC(kernel='linear', C=1)),
    ('K-Nearest Neighbors', KNeighborsClassifier(n_neighbors=5)),
    ('Naive Bayes', GaussianNB())
]

# Print evaluation results with improved formatting
for name, classifier in classifiers:
    # Fit the model on the training data
    classifier.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = classifier.predict(X_test)
    print(f"{name}")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy: {accuracy:.4f}")
    
    cm = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)
    print("-" * 100)

In [ ]:
# Cross-fold validation (need to explain what does this do)
for name, classifier in classifiers:
    # Create a KFold cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Perform cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=kf)
    
    # Print model name and cross-validation scores
    print(name)
    
    # Calculate and print the mean and standard deviation of the cross-validation scores
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    print(f"Mean Accuracy: {mean_score:.4f}")
    print(f"Standard Deviation: {std_score:.4f}")
    
    # Print a separator for better readability
    print("-" * 100)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors':np.arange(1,50),
    "weights": ["uniform", "distance"]
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

print("Best Score:", str(grid_search.best_score_))
print("Best Parameters: ", str(grid_search.best_params_))

In [ ]:
n = 38
knn = KNeighborsClassifier(n_neighbors=n, weights='distance')
knn.fit(X_train,y_train)
knn_accuracy = knn.score(X_test, y_test)
print(f"KNN({n}) accuracy: {knn_accuracy}")

### XGBoost, CatBoost, and LightGBM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from IPython.display import clear_output

# Load the Pima Indian Diabetes dataset
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop("Outcome", axis=1), df["Outcome"], test_size=0.2)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

xgboost = XGBClassifier()

catboost = CatBoostClassifier()

lightgbm = LGBMClassifier()

voting_classifier = VotingClassifier(
    estimators=[("xgboost", xgboost), ("catboost", catboost), ("lightgbm", lightgbm)], 
    voting="hard",
    weights=[2,1,3]
)


# Train the models
xgboost.fit(X_train, y_train)
catboost.fit(X_train, y_train)
lightgbm.fit(X_train, y_train)
voting_classifier.fit(X_train, y_train)
clear_output()

# Evaluate the models on the test set
xgboost_accuracy = xgboost.score(X_test, y_test)
catboost_accuracy = catboost.score(X_test, y_test)
lightgbm_accuracy = lightgbm.score(X_test, y_test)
voting_classifier_accuracy = voting_classifier.score(X_test, y_test)

print("XGBoost accuracy:", xgboost_accuracy)
print("CatBoost accuracy:", catboost_accuracy)
print("LightGBM accuracy:", lightgbm_accuracy)
print("Voting Classifier accuracy:", voting_classifier_accuracy)

In [ ]:
# GridSearchCV for XGBoost, CatBoost, and LightGBM

from sklearn.model_selection import GridSearchCV

# Load the Pima Indian Diabetes dataset
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop("Outcome", axis=1), df["Outcome"], test_size=0.2)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the XGBoost classifier
xgboost = XGBClassifier()

# Create the CatBoost classifier
catboost = CatBoostClassifier()

# Create the LightGBM classifier
lightgbm = LGBMClassifier()

# Define the hyperparameters to optimize
xgboost_params = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
}

catboost_params = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "depth": [3, 5, 7],
}

lightgbm_params = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
}

# Create the grid search objects
xgboost_grid = GridSearchCV(xgboost, xgboost_params, cv=5)
catboost_grid = GridSearchCV(catboost, catboost_params, cv=5)
lightgbm_grid = GridSearchCV(lightgbm, lightgbm_params, cv=5)

# Fit the grid search objects
xgboost_grid.fit(X_train, y_train)
clear_output()
catboost_grid.fit(X_train, y_train)
clear_output()
lightgbm_grid.fit(X_train, y_train)
clear_output()


print("XGBoost best parameters:", xgboost_grid.best_params_)
print("CatBoost best parameters:", catboost_grid.best_params_)
print("LightGBM best parameters:", lightgbm_grid.best_params_)

# Deep Learning Model

In [ ]:
from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D, BatchNormalization, LeakyReLU
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.regularizers import l2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x = df.drop(columns=['Outcome'],axis=1)
y = df['Outcome']

scaler = StandardScaler()
x = scaler.fit_transform(x)

x = x.reshape(x.shape[0], x.shape[1], 1)
y = y.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [ ]:
# Model Creation
model = Sequential()
model.add(InputLayer(input_shape=(8,)))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation= 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

In [ ]:
# Testing Model
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1],1)))
# model.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(16, activation= LeakyReLU(alpha=0.01), 
                kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu',
               kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
from IPython.display import clear_output
from keras.optimizers import Adam, RMSprop

adam = Adam(learning_rate = 0.001)
rmsprop = RMSprop(learning_rate = 0.001)

model.compile(
    loss = 'binary_crossentropy',
    optimizer = adam,
    metrics = ['accuracy']
)

hist = model.fit(
    x_train, y_train,
    epochs=150,
    batch_size=10,
    validation_data=(x_test, y_test),
    verbose=2
)
clear_output()

_, score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score)

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

# Summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()